In [1]:
%cd ..
import numpy as np
from skeletrack.io import read_skeleton_file
body_info = read_skeleton_file('dataset/raw/clapping/S001C001P001R001A010.skeleton')

/beegfs/xg703/Vision2018-RL


# Preprocessing

In [2]:
from skeletrack.preprocessing import preprocess
preprocess(body_info, gamma=0.95, is_clapping=True)

In [3]:
body_info[0][0].joints[
    ['v1','v2','v3','d1','d2','d3','a1','a2','a3']
].head()

,v1,v2,v3,d1,d2,d3,a1,a2,a3
0,1.008033,-0.122525,-0.059198,0.0,0.0,0.0,-0.000023,-0.000325,0.000022
1,0.501079,-0.065888,-0.041108,0.0,0.0,0.0,-0.000002,-0.000177,0.000039
2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,-0.265429,-0.065333,0.018349,0.0,0.0,0.0,0.000179,0.000143,-0.000244
4,0.182357,-0.319248,-0.037038,0.0,0.0,0.0,0.000680,-0.000056,-0.000322


In [4]:
body_info[0][0].weight

0.02903546361765786

## Sanity checks

### 1. Coordinate transformation: special joints

In [5]:
for i in range(len(body_info)):
    for j in range(len(body_info[0])):
        joints = body_info[i][j].joints
        neck = joints.loc[2][['v1','v2','v3']]
        right_hip = joints.loc[16][['v1','v2','v3']]
        assert np.all(np.isclose(neck, np.zeros(3)))
        assert np.all(np.isclose(right_hip, np.array([1., 0., 0.])))

### 2. Displacements

In [6]:
for i in range(1, len(body_info)):
    for j in range(len(body_info[0])):
        vprev = body_info[i-1][j].joints[['v1', 'v2', 'v3']]
        vthis = body_info[i][j].joints[['v1', 'v2', 'v3']]
        dthis = body_info[i][j].joints[['d1', 'd2', 'd3']]
        assert np.all(np.isclose(vthis - vprev, dthis))

### 3. Actions

In [7]:
for i in range(1, len(body_info) - 1):
    for j in range(len(body_info[0])):
        vprev = body_info[i-1][j].joints[['v1', 'v2', 'v3']]
        vthis = body_info[i][j].joints[['v1', 'v2', 'v3']]
        vnext = body_info[i+1][j].joints[['v1', 'v2', 'v3']]
        athis = body_info[i][j].joints[['a1', 'a2', 'a3']]
        assert np.all(np.isclose(vnext + vprev - 2*vthis, athis))

# Build the feature and label tensors

In [8]:
from skeletrack.preprocessing import extract_feature_labels

In [9]:
X, Y = extract_feature_labels(body_info)

In [10]:
print(f"X.shape = {X.shape}")
print(f"Y.shape = {Y.shape}")

X.shape = (69, 78)
Y.shape = (69, 40)


In [11]:
x = X[0, :]
y = Y[0, :]
print("x = ", x)
print("y = ", y)

x =  [-0.26542919 -0.06533322  0.01834924  0.          0.          0.
  0.18235696 -0.31924757 -0.0370382   0.          0.          0.
  0.56655302 -0.16769241  0.30908986  0.          0.          0.
  0.21907873 -0.19482129  0.16491655  0.          0.          0.
  0.12342204 -0.17354941  0.07058282  0.          0.          0.
  0.21987809  0.24196473  0.05240719  0.          0.          0.
  0.60338803  0.3912297   0.0186046   0.          0.          0.
  0.59726128 -0.00716272  0.32142488  0.          0.          0.
  0.57442014 -0.12487792  0.37560585  0.          0.          0.
 -0.00238041 -0.16161335  0.06582628  0.          0.          0.
  0.13784109 -0.26149559  0.05393149  0.          0.          0.
  0.57010741 -0.18348408  0.44874836  0.          0.          0.
  0.53283577 -0.07650273  0.43608627  0.          0.          0.        ]
y =  [ 1.79362565e-04  1.42531674e-04 -2.44409773e-04  6.80003501e-04
 -5.59703211e-05 -3.21564083e-04  3.98496407e-04  2.38394482e-05
 -1.58

Make sure these are correct:

In [12]:
body = body_info[0][0]
print("body.weight = ", body.weight)

body.weight =  0.02903546361765786


In [13]:
body.joints[
    ['v1','v2','v3','d1','d2','d3','a1','a2','a3']
]

,v1,v2,v3,d1,d2,d3,a1,a2,a3
0,1.008033,-1.225247e-01,-5.919816e-02,0.0,0.0,0.0,-0.000023,-3.253120e-04,2.229839e-05
1,0.501079,-6.588768e-02,-4.110765e-02,0.0,0.0,0.0,-0.000002,-1.766835e-04,3.916164e-05
2,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00
3,-0.265429,-6.533322e-02,1.834924e-02,0.0,0.0,0.0,0.000179,1.425317e-04,-2.444098e-04
4,0.182357,-3.192476e-01,-3.703820e-02,0.0,0.0,0.0,0.000680,-5.597032e-05,-3.215641e-04
5,0.566553,-1.676924e-01,3.090899e-01,0.0,0.0,0.0,0.000398,2.383945e-05,-1.589066e-04
6,0.219079,-1.948213e-01,1.649166e-01,0.0,0.0,0.0,0.000332,-1.106873e-04,1.804186e-04
7,0.123422,-1.735494e-01,7.058282e-02,0.0,0.0,0.0,0.005137,7.053422e-03,1.643979e-02
8,0.219878,2.419647e-01,5.240719e-02,0.0,0.0,0.0,-0.000363,-3.145504e-04,-1.169385e-03
9,0.603388,3.912297e-01,1.860460e-02,0.0,0.0,0.0,-0.000451,3.616950e-03,-4.366160e-03



# Batch preprocessing

In [14]:
from pathlib import Path
from progressbar import progressbar
Npositives = 200
Nnegatives = 200
np.random.seed(0)

## 1. Preprocess positive (clapping) examples

In [15]:
path = Path('dataset/raw/clapping/')
skfiles = np.array(list(path.iterdir()))
np.random.shuffle(skfiles)
skfiles = skfiles[:Npositives]

In [16]:
Xs = []
Ys = []
for skfile in progressbar(skfiles):
    try: # some files contain appearing/disappearing bodies and should be discarded
        bodyinfo = read_skeleton_file(skfile)
        preprocess(bodyinfo, gamma=0.95, is_clapping=True)
        X, Y = extract_feature_labels(bodyinfo)
        Xs.append(X)
        Ys.append(Y)
    except:
        pass
X = np.vstack(Xs)
Y = np.vstack(Ys)

100% (200 of 200) |######################| Elapsed Time: 0:01:23 Time:  0:01:23


In [17]:
np.save("dataset/processed/X_clapping.npy", X)
np.save("dataset/processed/Y_clapping.npy", Y)

## 2. Preprocess negative (non-clapping) examples

In [18]:
path = Path('dataset/raw/not-clapping/')
skfiles = np.array(list(path.iterdir()))
np.random.shuffle(skfiles)
skfiles = skfiles[:Nnegatives]

In [19]:
Xs = []
Ys = []
for skfile in progressbar(skfiles):
    try: # some files contain appearing/disappearing bodies and should be discarded
        bodyinfo = read_skeleton_file(skfile)
        preprocess(bodyinfo, gamma=0.95, is_clapping=False)
        X, Y = extract_feature_labels(bodyinfo)
        Xs.append(X)
        Ys.append(Y)
    except:
        pass
X = np.vstack(Xs)
Y = np.vstack(Ys)

100% (200 of 200) |######################| Elapsed Time: 0:01:54 Time:  0:01:54


In [20]:
np.save("dataset/processed/X_not_clapping.npy", X)
np.save("dataset/processed/Y_not_clapping.npy", Y)